In [ ]:
from grid import Grid
import graphviz

In [42]:
def get_moves(g:Grid):
    node = g.get_node_by_value('#')
    if node != None:
        m = []
        if node.up:
            m.append('up')
        if node.down:
            m.append('down')
        if node.right:
            m.append('right')
        if node.left:
            m.append('left')
        return m
   
def create_states(g:Grid):
    inputs = get_moves(g)
    outputs = []
    _g = Grid(3)
    for i in inputs:
        _g.assign_values(g.grid_to_array())
        _g.move_value('#', i)
        outputs.append(_g.grid_to_array())

    return outputs

def verify(a, b):
    match = 0
    for _a, _b in zip(a,b):
        if _a == _b:
            match +=1
    return (9-match)

def insort(h, states):
    for j in range(1, len(h)):
        key = h[j]
        _key = states[j]
        i = j - 1
        while i >= 0 and h[i] > key:
            h[i + 1] = h[i]
            states[i + 1] = states[i]
            i -= 1
        h[i + 1] = key
        states[i + 1] = _key
    return h, states

def format_state(state):
    result = ''
    for i in range(3):
        result += f'{state[i * 3: (i + 1) * 3]}\n'

    return result
        

def get_ascii(n):
    return chr(n)

In [43]:
grid = Grid(3)

x0 =['2', '8', '3',
     '1', '6', '4',
     '7', '#' ,'5']

t = ['1', '2', '3',
     '8', '#', '4',
     '7', '6' ,'5']

grid.assign_values(x0)
pending = []
checked = []

max_depth = 700


heuristics = []
to_delete = []
depth = 0
depth_acc = 0
last_heuristic = verify(x0, t)
alt_last_heuristic = last_heuristic + 1
current_state = x0



for i in range(max_depth):
    if current_state == t:
        print(f'\nTerminal: \n{format_state(current_state)} Depth: {depth}')
        break
    else:
        checked.append(current_state)
        print(f'================Depth: {depth}================')
        print(f'Current: {last_heuristic}\n{format_state(current_state)}\n')

        new_states = create_states(grid)
        print(f'Current:   {current_state}\n')
        for ns in new_states:
            print(f'New state: {ns}')
            if ns==current_state:
                to_delete.append(ns)
                print('Removed by equal to previous...\n')
            elif ns in pending:
                to_delete.append(ns)
                print('Removed by pending...\n')
            elif ns in checked:
                to_delete.append(ns)
                print('Removed by checked...\n')
            else:
                print(f'State ok\n')

        for td in to_delete:
            print(f'Deleting: {td}')
            new_states.remove(td)
        to_delete = []

        if len(new_states)==0:
            print(f'Pending. {len(pending)} left')
            depth -= depth_acc
            depth_acc = 0
            current_state = pending.pop(0)

            
        else:
            to_delete = []
            smallest = alt_last_heuristic + 1
            if len(new_states) > 1:
                print(f'Next available:\n')
                for ns in new_states:
                    
                    h = (verify(ns, t)) + (depth)

                    if h <= smallest:
                        smallest = h
                                    
                    print(f'Heuristic: {h}\n{format_state(ns)}\n')
                    heuristics.append(h)
                
                if smallest <= alt_last_heuristic:

                    heuristics, new_states = insort(heuristics, new_states)
                    current_state = new_states.pop(0)
                    last_heuristic = heuristics.pop(0) 
                    alt_last_heuristic = heuristics.pop(0)
                    
                    for ns, h in zip(new_states, heuristics):
                        pending.append(ns)
                        depth_acc = 0
                else:
                    current_state = pending.pop(0)
                    depth -= depth_acc
                    depth_acc = 0
                    print(f'Taken from pending {current_state}')

            else:
                current_state = new_states.pop(0)

        grid.assign_values(current_state)
        depth += 1
        depth_acc += 1
        heuristics = []


================Depth: 0================
Current: 5
['2', '8', '3']
['1', '6', '4']
['7', '#', '5']


Current:   ['2', '8', '3', '1', '6', '4', '7', '#', '5']

New state: ['2', '8', '3', '1', '#', '4', '7', '6', '5']
State ok

New state: ['2', '8', '3', '1', '6', '4', '7', '5', '#']
State ok

New state: ['2', '8', '3', '1', '6', '4', '#', '7', '5']
State ok

Next available:

Heuristic: 3
['2', '8', '3']
['1', '#', '4']
['7', '6', '5']


Heuristic: 6
['2', '8', '3']
['1', '6', '4']
['7', '5', '#']


Heuristic: 6
['2', '8', '3']
['1', '6', '4']
['#', '7', '5']


================Depth: 1================
Current: 3
['2', '8', '3']
['1', '#', '4']
['7', '6', '5']


Current:   ['2', '8', '3', '1', '#', '4', '7', '6', '5']

New state: ['2', '#', '3', '1', '8', '4', '7', '6', '5']
State ok

New state: ['2', '8', '3', '1', '6', '4', '7', '#', '5']
Removed by checked...

New state: ['2', '8', '3', '1', '4', '#', '7', '6', '5']
State ok

New state: ['2', '8', '3', '#', '1', '4', '7', '6', '5']
St